In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Importing the libraries
import numpy as np
import pandas as pd
! pip install scikit-surprise
from sklearn import preprocessing
from collections import defaultdict
from surprise import SVD
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

# Suppressing Warnings
import warnings
warnings.filterwarnings('ignore')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
open1=pd.read_csv('phone_user_review_file_1.csv',encoding='latin-1')
open1.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [4]:
open2=pd.read_csv('phone_user_review_file_2.csv',encoding='latin-1')
open2.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/leagoo-lead-7/,4/15/2015,en,us,Amazon,amazon.com,2.0,10.0,"The telephone headset is of poor quality , not...",luis,Leagoo Lead7 5.0 Inch HD JDI LTPS Screen 3G Sm...
1,/cellphones/leagoo-lead-7/,5/23/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,This is my first smartphone so I have nothing ...,Mark Lavin,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
2,/cellphones/leagoo-lead-7/,4/27/2015,en,gb,Amazon,amazon.co.uk,8.0,10.0,Great phone. Battery life not great but seems ...,tracey,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
3,/cellphones/leagoo-lead-7/,4/22/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,Best 90 quid I've ever spent on a smart phone,Reuben Ingram,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...
4,/cellphones/leagoo-lead-7/,4/18/2015,en,gb,Amazon,amazon.co.uk,10.0,10.0,I m happy with this phone.it s very good.thx team,viorel,Leagoo Lead 7 Lead7 MTK6582 Quad core 1GB RAM ...


In [5]:
open3=pd.read_csv('phone_user_review_file_3.csv',encoding='latin-1')
open3.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,11/7/2015,pt,br,Submarino,submarino.com.br,6.0,10.0,"recomendo, eu comprei um, a um ano, e agora co...",herlington tesch,Samsung Smartphone Samsung Galaxy S3 Slim G381...
1,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,10/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,Comprei um pouco desconfiada do site e do celu...,Luisa Silva Marieta,Samsung Smartphone Samsung Galaxy S3 Slim G381...
2,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,10.0,10.0,"Muito bom o produto, obvio que tem versÃµes me...",Cyrus,Samsung Smartphone Samsung Galaxy S3 Slim G381...
3,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/2/2015,pt,br,Submarino,submarino.com.br,8.0,10.0,Unica ressalva fica para a camera que poderia ...,Marcela Santa Clara Brito,Samsung Smartphone Samsung Galaxy S3 Slim G381...
4,/cellphones/samsung-galaxy-s-iii-slim-sm-g3812/,9/1/2015,pt,br,Colombo,colombo.com.br,8.0,10.0,Rapidez e atenÃ§Ã£o na entrega. O aparelho Ã© ...,Claudine Maria Kuhn Walendorff,"Smartphone Samsung Galaxy S3 Slim, Dual Chip, ..."


In [6]:
open4=pd.read_csv('phone_user_review_file_4.csv',encoding='latin-1')
open4.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-s7262-duos-galaxy-ace/,3/11/2015,en,us,Amazon,amazon.com,2.0,10.0,was not conpatable with my phone as stated. I ...,Frances DeSimone,Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce...
1,/cellphones/samsung-s7262-duos-galaxy-ace/,17/11/2015,en,in,Zopper,zopper.com,10.0,10.0,Decent Functions and Easy to Operate Pros:- Th...,Expert Review,Samsung Galaxy Star Pro S7262 Black
2,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,4.0,10.0,Not Good Phone such price. Hang too much and v...,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
3,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,6.0,10.0,not bad for features,Amazon Customer,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)
4,/cellphones/samsung-s7262-duos-galaxy-ace/,29/10/2015,en,in,Amazon,amazon.in,10.0,10.0,Excellent product,NHK,Samsung Galaxy Star Pro GT-S7262 (Midnight Black)


In [7]:
open5=pd.read_csv('phone_user_review_file_5.csv',encoding='latin-1')
open5.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,2.0,10.0,I bought 1 month before. currently speaker is ...,venkatesh,Karbonn K1616
1,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,6.0,10.0,"I just bought one week back, I have Airtel con...",Venkat,Karbonn K1616
2,/cellphones/karbonn-k1616/,7/13/2016,en,in,91 Mobiles,91mobiles.com,4.0,10.0,one problem in this handset opera is not worki...,krrish,Karbonn K1616
3,/cellphones/karbonn-k1616/,4/25/2014,en,in,Naaptol,naaptol.com,10.0,10.0,here Karbonn comes up with an another excellen...,BRIJESH CHAUHAN,Karbonn K1616 - Black
4,/cellphones/karbonn-k1616/,4/23/2013,en,in,Naaptol,naaptol.com,10.0,10.0,"What a phone, all so on Naaptol my god 23% off...",Suraj CHAUHAN,Karbonn K1616 - Black


In [8]:
open6=pd.read_csv('phone_user_review_file_6.csv',encoding='latin-1')
open6.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-instinct-sph-m800/,9/16/2011,en,us,Phone Arena,phonearena.com,8.0,10.0,I've had the phone for awhile and it's a prett...,ajabrams95,Samsung Instinct HD
1,/cellphones/samsung-instinct-sph-m800/,2/13/2014,en,us,Amazon,amazon.com,6.0,10.0,to be clear it is not the sellers fault that t...,Stephanie,Samsung SPH M800 Instinct
2,/cellphones/samsung-instinct-sph-m800/,12/30/2011,en,us,Phone Scoop,phonescoop.com,9.0,10.0,Well i love this phone. i have had ton of phon...,snickers,Instinct M800
3,/cellphones/samsung-instinct-sph-m800/,10/18/2008,en,us,HandCellPhone,handcellphone.com,4.0,10.0,I have had my Instinct for several months now ...,A4C,Samsung Instinct
4,/cellphones/samsung-instinct-sph-m800/,9/6/2008,en,us,Reviewed.com,reviewed.com,6.0,10.0,i have had this instinct phone for about two m...,betaBgood,Samsung Instinct


In [9]:
#  Merge the provided CSVs into one data-frame. 
data=pd.concat([open1,open2,open3,open4,open5,open6],axis=0)
data.head()

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [10]:
# Explore, understand the Data and share at least 2 observations. 
data.shape

(1415133, 11)

In [11]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
score,1351644.0,8.00706,2.616121,0.2,7.2,9.2,10.0,10.0
score_max,1351644.0,10.00000,0.000000,10.0,10.0,10.0,10.0,10.0


In [12]:
data.dtypes

phone_url     object
date          object
lang          object
country       object
source        object
domain        object
score        float64
score_max    float64
extract       object
author        object
product       object
dtype: object

In [13]:
#Check for missing values.
data.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        63489
score_max    63489
extract      19361
author       63202
product          1
dtype: int64

In [14]:
# Impute the missing values
data = data.fillna(data.median())


In [15]:
data =data.dropna()

In [16]:
data.isnull().values.any()

False

In [17]:
# Check for duplicate values and remove them
data.duplicated().sum()

4809

In [18]:
data = data.drop_duplicates()

In [19]:
# Round oﬀ scores to the nearest integers. 
data['score'] = data['score'].astype(int) 
data['score_max'] = data['score_max'].astype(int)

In [20]:
# Check for duplicate values and remove them
data.drop(columns=['phone_url','date','lang','country','source','domain','score_max','extract'], axis = 1, inplace = True)

In [21]:
review = data.copy()

In [22]:
review.shape

(1331607, 3)

In [23]:
# Keep only 1000000 data samples. Use random state=612
rev = review.sample(n=1000000, random_state=612)

In [24]:
# Identify the most rated features.
# sorting on products that got highest mean score
rev.groupby('product')['score'].mean().sort_values(ascending=False).head()  

product
LG Electronics G3S UK SIM-Free Smartphone - Shine Gold                                                                                                               10.0
TTfone Pluto (TT600) Pay As You Go - Prepay - PAYG - Big Button Clamshell Flip Senior Emergency Mobile Phone - Easy to Use Simple (Orange with Â£10 Credit, Blue)    10.0
BlackBerry Curve 9300 Pink Simfree Unlocked Mobile Phone                                                                                                             10.0
BlackBerry Curve 9300 Zwart                                                                                                                                          10.0
TTfone Pluto (TT600) Pay As You Go - Prepay - PAYG - Big Button Clamshell Flip Senior Emergency Mobile Phone - Easy to Use Simple (Virgin Pay as you go, Blue)       10.0
Name: score, dtype: float64

In [25]:
# 2 b. Identify the users with most number of reviews. 
(rev['author'].value_counts()).head()

Amazon Customer    57792
Cliente Amazon     14624
e-bit               6317
Client d'Amazon     5698
Amazon Kunde        3574
Name: author, dtype: int64

In [26]:
# The product that got most number of reviews.
rev['product'].value_counts().head()

Lenovo Vibe K4 Note (White,16GB)     3908
Lenovo Vibe K4 Note (Black, 16GB)    3234
OnePlus 3 (Graphite, 64 GB)          3128
OnePlus 3 (Soft Gold, 64 GB)         2643
Samsung Galaxy Express I8730         2012
Name: product, dtype: int64

In [27]:
# extracting authors who gave greater than 50 ratings
df = pd.DataFrame(columns=['author', 'a_count'])
df['author']=df['author'].value_counts().index.tolist() 
df['a_count'] = list(df['author'].value_counts() > 50)

In [28]:
# get names of indexes for which count column value is False
index_names = df[ df['a_count'] == False ].index 
# drop these row indexes from dataFrame 
df.drop(index_names, inplace = True) 
df

,author,a_count


In [29]:
# extracting product that got more than 50 ratings
df2 = pd.DataFrame(columns=['product', 'p_count'])
df2['product']=df2['product'].value_counts().index.tolist() 
df2['p_count'] = list(df2['product'].value_counts() > 50)

In [30]:
# get names of indexes for which count column value is False
index_names = df2[ df2['p_count'] == False ].index 
# drop these row indexes from dataFrame 
df2.drop(index_names, inplace = True)

# Build a popularity based model and recommend top 5 mobile phones. 

In [31]:
# calculating the mean score for a product.
top_phones = pd.DataFrame(rev.groupby('product')['score'].mean()) 

In [32]:
# calculating the number of ratings according to the costumers.
top_phones['rating_counts'] = pd.DataFrame(rev.groupby('product')['score'].count())  
top_phones.sample(5)

,score,rating_counts
product,,
"LG Revolution VS910 4G LTE phone Large 4.3-inch touch screen, 5-megapixel camera for Verizon",4.000000,13
Samsung Galaxy S III mini Value Edition,9.000000,2
Microsoft Lumia 950 wit / 32 GB,7.000000,7
Samsung Smartphone Dual Chip Samsung Galaxy S5 Duos SM-G900M Desbloqueado Android 4.4 4G 16MB MemÃ³ria 16GB GPS NFC Branco,10.000000,1
LG Optimus L5 E615 Black Dual Sim Android 4.0 ICS WiFI Unlocked Smart Phone,6.727273,11


In [33]:
# Recommending the 5 mobile phones based in highest mean score and highest number of ratings the product got. 
top_phones.sort_values(by=['score','rating_counts'], ascending=[False,False]).head()

,score,rating_counts
product,,
Samsung Galaxy Note5,10.0,144
Motorola Smartphone Motorola Moto X Desbloqueado Preto Android 4.2.2 CÃ¢mera 10MP e Frontal 2MP MemÃ³ria Interna de 16GB GSM,10.0,143
Nokia Smartphone Nokia Lumia 520 Desbloqueado Oi Preto Windows Phone 8 CÃ¢mera 5MP 3G Wi-Fi MemÃ³ria Interna 8G GPS,10.0,129
Motorola Smartphone Motorola Moto G Dual Chip Desbloqueado TIM Android 4.3 Tela 4.5 8GB 3G Wi-Fi CÃ¢mera 5MP - Preto,10.0,126
Motorola Smartphone Motorola Novo Moto G DTV Colors Dual Chip XT 1069 Desbloqueado Android 4.4 Tela 5 16GB 3G Wi-Fi CÃ¢mera de 8MP - Preto,10.0,119


# Build a collaborative filtering model using SVD. 

In [34]:
# arranging columns in the order of user id,item id and rating to be fed in the svd
columns_titles = ['author','product','score']
revs = review.reindex(columns=columns_titles)
revs.head()

,author,product,score
0,CarolAnn35,Samsung Galaxy S8,10
1,james0923,Samsung Galaxy S8,10
2,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",6
3,Buster2020,Samsung Galaxy S8 64GB (AT&T),9
4,S Ate Mine,Samsung Galaxy S8,4


In [35]:
# Keep only 5000 data samples. Use random state=612
newdf = revs.sample(n=5000, random_state=612)

In [36]:
# Build a collaborative filtering model using SVD. 
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(newdf,reader = reader)

In [37]:
trainset = data.build_full_trainset()

In [38]:
trainset

In [39]:
trainset.ur

defaultdict(list,
            {0: [(0, 10.0)],
             1: [(1, 10.0)],
             2: [(2, 9.0)],
             3: [(3, 4.0)],
             4: [(4, 9.0)],
             5: [(5, 10.0)],
             6: [(6, 2.0),
              (784, 8.0),
              (456, 10.0),
              (1886, 10.0),
              (1992, 6.0),
              (2862, 10.0),
              (3191, 10.0),
              (311, 8.0),
              (3475, 10.0),
              (3642, 10.0),
              (456, 2.0)],
             7: [(7, 10.0)],
             8: [(8, 8.0), (3484, 10.0)],
             9: [(9, 8.0)],
             10: [(10, 2.0)],
             11: [(11, 10.0),
              (16, 10.0),
              (26, 8.0),
              (36, 10.0),
              (43, 10.0),
              (48, 10.0),
              (49, 10.0),
              (59, 2.0),
              (26, 2.0),
              (129, 8.0),
              (159, 2.0),
              (174, 10.0),
              (203, 10.0),
              (203, 8.0),
              (

In [40]:
algo = SVD()
algo.fit(trainset)

In [41]:
# Than predict ratings for all pairs.
testset = trainset.build_anti_testset()

In [ ]:
pred = algo.test(testset)

In [ ]:
pred

Above are the  predicted items and their estimated ratings for test user.

In [ ]:
def get_top_n(pred, n=5):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in pred:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [ ]:
#  Try and recommend top 5 products for test users
top_n = get_top_n(pred, n=5)

In [ ]:
top_n 

Above are the top 5 predicted items and their ratings for test users.

In [ ]:
# Evaluate the collaborative model. Print RMSE value for SVD
print("SVD Model : Test Set")
accuracy.rmse(pred, verbose=True)

# Build a collaborative filtering model using kNNWithMeans from surprise using Item based model

In [ ]:
newdf2 = revs.sample(n=5000, random_state=612)

In [ ]:
reader = Reader(rating_scale=(1, 10))
data_I = Dataset.load_from_df(newdf2,reader = reader)

In [ ]:
trainset_I, testset_I = train_test_split(data_I, test_size=.30)

In [ ]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo_I = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo_I.fit(trainset_I)

In [ ]:
# run the  model against the testset
test_pred = algo.test(testset_I)

In [ ]:
test_pred

In [ ]:
# get RMSE
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

# Build a collaborative filtering model using kNNWithMeans from surprise using User based model

In [ ]:
reader = Reader(rating_scale=(1, 10))
data_U = Dataset.load_from_df(newdf2,reader = reader)

In [ ]:
trainset_U, testset_U = train_test_split(data_U, test_size=.30)

In [ ]:
# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset_U)

when, author = Frances DeSimone ,
item: Samsung Galaxy Star Pro DUOS S7262 Unlocked Ce.
estimated rating is 8.03

In [ ]:
# run the trained model against the testset
test_pred_U = algo.test(testset_U)

In [ ]:
# Predict score (average rating) for test users
test_pred_U

Above are the prediction of user item combinations and the estimated ratings.

In [ ]:
#  Evaluate the collaborative model. Print RMSE value for User Based CF
print("User-based Model : Test Set")
accuracy.rmse(test_pred_U, verbose=True)


Popularity based recommendation system relies on the popularity, frequency counts of which items were most purchased.  
As the name suggests Popularity based recommendation system works with the trend. It basically uses the items which are in trend right now. For example, if any product which is usually bought by every new user then there are chances that it may suggest that item to the user who just signed up.andby Google News other news websites to show Top Stories with images.


Collaborative filtering is a technique that can filter out items that a user might like on the basis of reactions by similar users. It works by searching a large group of people and finding a smaller set of users with tastes similar to a particular user.t isa personalised recommender system , recommendations are made based on the past behaviour of the user. Most websites like Amazon, YouTube, and Netflix use collaborative filtering as a part of their sophisticated recommendation system

 Apart from Popularity and Collaborative Filtering , Content-based, Demographic, Utility based, Knowledge based and Hybrid recommendation system can be used as per the user needs.